In [24]:
import cv2
import numpy as np
import boto
import yaml
import time

In [44]:
def logger(log):
    with open("logfile.txt","a") as logfile:
        logfile.write(",".join([log,time.strftime("%m/%d/%Y %H:%M")]))
        logfile.close()

In [17]:
#send the frame to an S3 instance
from boto.s3.connection import S3Connection

myKeys = yaml.load(open('libraryKeys.json', 'r'))
AWSAccessKeyId=myKeys['key']
AWSSecretKey=myKeys['secret']
conn = S3Connection(AWSAccessKeyId, AWSSecretKey)

In [19]:
# Capture one frame and save it to file
def capture():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cv2.imwrite("frame.jpg", frame)
    cap.release()

In [31]:
capture()

In [22]:
#Upload the file to bucket
def send():
    mybucket = conn.get_bucket('capitolhilltoollibrary',validate=False)
    myKey = boto.s3.key.Key(mybucket)
    myKey.key = "whats_happening_at_the_library.jpg"
    myKey.set_contents_from_file(open('frame.jpg', 'rb'))
    myKey.set_metadata('upload time',time.strftime("%m/%d/%Y %H:%M"))
    #change the file to be public
    myKey.set_acl("public-read")
    #add date
    myKey.set_metadata('upload time',time.strftime("%m/%d/%Y %H:%M"))

In [30]:
send()

In [28]:
capture()
send()

In [46]:
#loop that takes the picture and sends it
while True:
    capture()
    send()
    logger("photo taken and sent")
    time.sleep(60)

09/28/2017 01:29
09/28/2017 01:30
09/28/2017 01:31
09/28/2017 01:32


KeyboardInterrupt: 